# Paper Review 

#### This sentiment analysis data set contains scientific paper reviews from an international conference on computing and informatics. The task is to predict the orientation or the evaluation of a review.

### Import required libraries 

In [1]:
#review
import pandas as pd
import numpy as np
import os
import re

###  set default encoding method

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

###  Looking output of Multiple commands of single cell 

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Load JSON file and convert unstructred data into structured format

In [4]:
import json

f1 = open('reviews.json')
new_data = json.load(f1)
f1.close()

 f1 = open("MainData.csv","w+")
 header = "id" + "," + "Decision" + "," + "Confidence" + "," + "Evaluation" + "," + "Sub-id" + "," 
 header = header +  "Lan" + "," + "Orientation" + "," + "Text" + "," + "TimeSpan" + "\n"

 f1.write(header)

 allData = new_data['paper']
 newDict = {}

 for i in range(len(allData)):
     dataDict = allData[i]
     paper_id = dataDict['id']
     target = dataDict['preliminary_decision']
     reviewStruct = dataDict['review']
      
     for review in reviewStruct: 
         dataDict = str(paper_id) + "," + target + ","
         for reviewDetail in review:
             dataDict = dataDict + str(review['confidence']) + ","
             dataDict = dataDict + str(review['evaluation']) + ","
             dataDict = dataDict + str(review['id']) + ","
             dataDict = dataDict + str(review['lan']) + ","
             dataDict = dataDict + str(review['orientation']) + ","
             dataDict = dataDict + str(review['text']).replace("," , "|") + ","
             dataDict = dataDict + str(review['timespan'])
             break
         f1.write(dataDict + "\n")

 f1.close()

### Load the csv file into enviornment

In [5]:
text_data = pd.read_csv("MainData.csv")

### Perform some summary statistics on data

In [6]:
text_data.head(5)
text_data.tail(5)

,id,Decision,Confidence,Evaluation,Sub-id,Lan,Orientation,Text,TimeSpan
0,1,accept,4,1,1,es,0,- El artículo aborda un problema contingente y...,2010-07-05
1,1,accept,4,1,2,es,1,El artículo presenta recomendaciones prácticas...,2010-07-05
2,1,accept,5,1,3,es,1,- El tema es muy interesante y puede ser de mu...,2010-07-05
3,2,accept,4,2,1,es,1,Se explica en forma ordenada y didáctica una e...,2010-07-05
4,2,accept,4,2,2,es,0,NaN,2010-07-05


,id,Decision,Confidence,Evaluation,Sub-id,Lan,Orientation,Text,TimeSpan
400,170,reject,5,-1,1,es,-2,El trabajo pretende ofrecer una visión del uso...,2015-07-05
401,170,reject,4,-1,2,es,-1,El paper está bien escrito y de fácil lectura....,2015-07-05
402,171,accept,4,-1,1,es,0,Observación de fondo: No se presenta un ejemp...,2015-07-05
403,171,accept,3,1,2,es,-1,Se propone un procedimiento para elaborar máqu...,2015-07-05
404,172,no decision,4,1,1,es,-1,El artículo describe básicamente los component...,2015-07-05


### Finding Missing Values

In [7]:
text_data.isnull().sum()

id             0
Decision       0
Confidence     0
Evaluation     0
Sub-id         0
Lan            0
Orientation    0
Text           6
TimeSpan       0
dtype: int64

### Droping the missing values

In [8]:
text_data.dropna(inplace=True)
text_data.isnull().sum()

id             0
Decision       0
Confidence     0
Evaluation     0
Sub-id         0
Lan            0
Orientation    0
Text           0
TimeSpan       0
dtype: int64

In [9]:
# Print the unique classes and their counts/frequencies
classes = np.unique(text_data['Decision'], return_counts=True) # np.unique returns a tuple with class names and counts
#print(classes[0]) #Print the list of unique classes
#print(classes[1]) #Print the frequency of each classes
classes

(array(['accept', 'no decision', 'probably reject', 'reject'], dtype=object),
 array([257,   1,  19, 122], dtype=int64))

### As we analyze the frequency of the different classes, 'no decision' should be comes 1 so we replace by 'reject'

In [10]:
text_data = text_data.replace('no decision', 'reject')
text_data.tail()

,id,Decision,Confidence,Evaluation,Sub-id,Lan,Orientation,Text,TimeSpan
400,170,reject,5,-1,1,es,-2,El trabajo pretende ofrecer una visión del uso...,2015-07-05
401,170,reject,4,-1,2,es,-1,El paper está bien escrito y de fácil lectura....,2015-07-05
402,171,accept,4,-1,1,es,0,Observación de fondo: No se presenta un ejemp...,2015-07-05
403,171,accept,3,1,2,es,-1,Se propone un procedimiento para elaborar máqu...,2015-07-05
404,172,reject,4,1,1,es,-1,El artículo describe básicamente los component...,2015-07-05


## Convert Text into Term Document and Count Vectorization

###  Extract text column from the dataset and apply preprocessing steps on it....

In [11]:
txt_col = text_data['Text']
txt_col.head()

0    - El artÃ­culo aborda un problema contingente y...
1    El artÃ­culo presenta recomendaciones prÃ¡cticas...
2    - El tema es muy interesante y puede ser de mu...
3    Se explica en forma ordenada y didÃ¡ctica una e...
5    Los autores describen una metodologÃ­a para des...
Name: Text, dtype: object

In [12]:
txt = txt_col.tolist()

In [13]:
type(txt)
txt[0]
len(txt)

list

'- El art\xc3\xadculo aborda un problema contingente y muy relevante| e incluye tanto un diagn\xc3\xb3stico nacional de uso de buenas pr\xc3\xa1cticas como una soluci\xc3\xb3n (buenas pr\xc3\xa1cticas concretas). - El lenguaje es adecuado.  - El art\xc3\xadculo se siente como la concatenaci\xc3\xb3n de tres art\xc3\xadculos diferentes: (1) resultados de una encuesta| (2) buenas pr\xc3\xa1cticas de seguridad| (3) incorporaci\xc3\xb3n de buenas pr\xc3\xa1cticas. - El orden de las secciones ser\xc3\xada mejor si refleja este orden (la versi\xc3\xb3n revisada es #2| #1| #3). - El art\xc3\xadculo no tiene validaci\xc3\xb3n de ning\xc3\xban tipo| ni siquiera por evaluaci\xc3\xb3n de expertos.'

399

In [14]:
#txt[330]
txt_dataframe = pd.DataFrame(txt)

In [15]:
new_txt = txt_dataframe.drop(txt_dataframe.index[321:331])
text_data = text_data.drop(text_data.index[321:331])

In [16]:
len(new_txt)
txt_new_data = new_txt[0].tolist()
type(txt_new_data)

389

list

###  Translated object is not iterated so we need to do manually for all rows... First for 1:100 rows

In [17]:
from googletrans import Translator
translator = Translator()

In [18]:
t = translator.translate(txt_new_data, dest='en')
t_csv = pd.DataFrame(t)
t_csv.head()
pd.DataFrame.to_csv(t_csv, "Text.csv", index= False)

In [19]:
txt_frame = pd.read_csv("Text.csv")

In [20]:
txt_frame.columns = ["Text"]
txt_frame.head()

,Text
0,- The article addresses a contingent and very ...
1,The article presents practical recommendations...
2,- The topic is very interesting and a guide to...
3,An experience of ICT use for academic collabor...
4,The authors describe a methodology to develop ...


In [21]:
txt_frame1 = txt_frame["Text"].tolist()

In [22]:
#Some text preprocessing- Removing extra spaces and numbers

for i in range(len(txt_frame1)):
    txt_frame1[i]=re.sub(r'\s+', ' ', txt_frame1[i]) #Removing white spaces in the sentence
    txt_frame1[i]=re.sub(r'\d+', ' ', txt_frame1[i]) #Removing white spaces in the sentenceb

In [23]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
#Lets consider TF-idf
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1),stop_words='english')
X_train_tfidf = tfidf_transformer.fit_transform(txt_frame1)
X_train_tfidf.shape
Dense_mat=X_train_tfidf.todense()
A=pd.DataFrame(Dense_mat,columns=tfidf_transformer.get_feature_names())

A.shape

(389, 4317)

(389, 4317)

In [24]:
txt_eval_col = text_data[["Confidence","Evaluation","Orientation"]]

In [25]:
new_main_data = pd.concat(axis=1,objs=[A,txt_eval_col])

In [26]:
text_data.columns

Index([u'id', u'Decision', u'Confidence', u'Evaluation', u'Sub-id', u'Lan',
       u'Orientation', u'Text', u'TimeSpan'],
      dtype='object')

## Model Building

###  Naive Bayes On entire data

In [27]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, text_data['Decision'])
pred = clf.predict(X_train_tfidf)

In [28]:
text_data['Decision'].value_counts()

accept             255
reject             115
probably reject     19
Name: Decision, dtype: int64

In [29]:
from sklearn.metrics import confusion_matrix, classification_report
prediction = classification_report(text_data['Decision'], pred)

In [30]:
prediction

'                 precision    recall  f1-score   support\n\n         accept       0.67      1.00      0.80       255\nprobably reject       0.00      0.00      0.00        19\n         reject       1.00      0.07      0.13       115\n\n    avg / total       0.73      0.68      0.56       389\n'

### Spliting the data into train and test

In [32]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split(Dense_mat, text_data["Decision"],test_size = 0.3, random_state = 5211)


### Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
log_model = logreg.fit(x_train,y_train)

###  Prediction on train and validation data

In [35]:
pred_train = log_model.predict(x_train)
pred_test = log_model.predict(x_test)

### Display the confusion matrix

In [42]:
### Train data
classification_report(y_train, pred_train) 

## Test Data
classification_report(y_test, pred_test) 

'                 precision    recall  f1-score   support\n\n         accept       0.76      1.00      0.86       172\nprobably reject       0.00      0.00      0.00        10\n         reject       1.00      0.51      0.68        90\n\n    avg / total       0.81      0.80      0.77       272\n'

'                 precision    recall  f1-score   support\n\n         accept       0.72      1.00      0.83        83\nprobably reject       0.00      0.00      0.00         9\n         reject       1.00      0.04      0.08        25\n\n    avg / total       0.72      0.72      0.61       117\n'

###  Naive Bayes on Splitted Data

In [44]:
naive_model = MultinomialNB().fit(x_train,y_train)
naive_train = naive_model.predict(x_train)
naive_test = naive_model.predict(x_test)

###  Predict on train and test data 

In [46]:
classification_report(y_train, naive_train)
classification_report(y_test, naive_test)

'                 precision    recall  f1-score   support\n\n         accept       0.67      1.00      0.81       172\nprobably reject       0.00      0.00      0.00        10\n         reject       1.00      0.19      0.32        90\n\n    avg / total       0.76      0.69      0.61       272\n'

'                 precision    recall  f1-score   support\n\n         accept       0.71      1.00      0.83        83\nprobably reject       0.00      0.00      0.00         9\n         reject       0.00      0.00      0.00        25\n\n    avg / total       0.50      0.71      0.59       117\n'

### Decision Tree 

In [50]:
from sklearn import tree
tree_clf = tree.DecisionTreeClassifier()
tree_model = tree_clf.fit(x_train, y_train)

### Predict on train and test

In [56]:
tree_train = tree_model.predict(x_train)
tree_test = tree_model.predict(x_test)

In [57]:
classification_report(y_train, tree_train)
classification_report(y_test, tree_test)

'                 precision    recall  f1-score   support\n\n         accept       1.00      1.00      1.00       172\nprobably reject       1.00      1.00      1.00        10\n         reject       1.00      1.00      1.00        90\n\n    avg / total       1.00      1.00      1.00       272\n'

'                 precision    recall  f1-score   support\n\n         accept       0.70      0.77      0.74        83\nprobably reject       0.00      0.00      0.00         9\n         reject       0.16      0.16      0.16        25\n\n    avg / total       0.53      0.58      0.56       117\n'

###  Support Vector Machine

In [60]:
from sklearn import svm
svm_clf = svm.SVC()
svm_model = svm_clf.fit(x_train, y_train)

In [62]:
svm_train = svm_model.predict(x_train)
svm_test = svm_model.predict(x_test)

In [64]:
classification_report(y_train, svm_train)
classification_report(y_test, svm_test)

'                 precision    recall  f1-score   support\n\n         accept       0.63      1.00      0.77       172\nprobably reject       0.00      0.00      0.00        10\n         reject       0.00      0.00      0.00        90\n\n    avg / total       0.40      0.63      0.49       272\n'

'                 precision    recall  f1-score   support\n\n         accept       0.71      1.00      0.83        83\nprobably reject       0.00      0.00      0.00         9\n         reject       0.00      0.00      0.00        25\n\n    avg / total       0.50      0.71      0.59       117\n'